In [2]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import tensorflow as tf
from lime import lime_image
from skimage.segmentation import mark_boundaries
import shap


c:\Users\wfaiz\anaconda3\envs\pcb_proj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
model_path = os.path.join(parent_dir, 'models', "model_enhanced_unet_100_2.keras")

In [4]:
# Load the model
model = tf.keras.models.load_model(model_path)

In [5]:
def crop_image_to_patches(image, crop_size):
    patches = []
    for i in range(0, image.shape[0], crop_size[0]):
        for j in range(0, image.shape[1], crop_size[1]):
            patch = image[i:i + crop_size[0], j:j + crop_size[1]]
            patches.append(patch)
    return patches

image_path = 'C:/users/wfaiz/Data_Science/PCB_Defect/data_full/Images/l_light_01_missing_hole_17_3_600.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
image = cv2.resize(image, (600, 600))

patches = crop_image_to_patches(image, (100, 100))

In [6]:
patches_rgb = [np.stack((patch,)*3, axis=-1) / 255.0 for patch in patches] # stacking gryscale image *3 to make psuedo RGB image


In [7]:
num_classes=7

In [8]:

# Assuming 'model' is your trained Keras model
input_shape = (100, 100, 1)  # Adjust if necessary

# Create a SHAP explainer
explainer = shap.GradientExplainer(model, np.zeros((1, *input_shape)))

# Select an image for explanation (example)
image = np.random.rand(1, 100, 100, 1)  # Replace with your actual image

# Compute SHAP values for segmentation and classification outputs
shap_values_segmentation_array = []
shap_values_classification_array = []
for patch_rgb in patches_rgb:
  
    shap_values_segmentation, shap_values_classification = explainer.shap_values(patch_rgb)
    shap_values_segmentation_array.append(shap_values_segmentation)
    shap_values_classification_array.append(shap_values_classification)



ValueError: could not broadcast input array from shape (100,100,3) into shape (100,3)

In [ ]:
# Plot SHAP values for the segmentation output
masks = []
for shap_seg in shap_values_segmentation_array:
    _, mask = explanation.get_image_and_mask(explanation.top_labels[0],
                                             positive_only=True,
                                             num_features=10,
                                             hide_rest=True)
    masks.append(mask)

In [ ]:

shap.image_plot(shap_values_segmentation, -image)

# Plot SHAP values for the classification output
for class_index in range(num_classes):
    shap.image_plot([shap_values_classification[class_index]], -image)
